# DBMS Lab Programs
Written by **Arnav Sharma@NIE**

### Experiment 1


Create a table called Employee & execute the following. 
Employee (EMPNO, ENAME, JOB, MANAGER_NO, SAL, COMMISSION) 
1. Create a user and grant all permissions to the user. 
2. Insert any three records in the employee table containing attributes. EMPNO, 
ENAME JOB, MANAGER_NO, SAL, COMMISSION and use rollback. Check 
the result. 
3. Add primary key constraint and not null constraint to the employee table. 
4. Insert NULL values to the employee table and verify the result. 

In [ ]:
-- 1. Create user and grant privileges
CREATE USER emp_user IDENTIFIED BY emp_pass;
GRANT ALL PRIVILEGES TO emp_user;

-- Connect as the new user
CONNECT emp_user/emp_pass;

-- Create Employee table
CREATE TABLE Employee (
  EMPNO      NUMBER,        -- employee number
  ENAME      VARCHAR2(50),  -- name
  JOB        VARCHAR2(30),  -- job title
  MANAGER_NO NUMBER,        -- manager’s EMPNO
  SAL        NUMBER(10,2),  -- salary
  COMMISSION NUMBER(10,2)   -- commission
);

-- Insert three rows, then rollback
SAVEPOINT before_inserts;
INSERT INTO Employee VALUES (101,'Alice','Developer',100,60000,1000);
INSERT INTO Employee VALUES (102,'Bob','Analyst',100,55000,500);
INSERT INTO Employee VALUES (103,'Charlie','Manager',NULL,75000,NULL);
SELECT * FROM Employee;  -- shows 3 rows
ROLLBACK TO before_inserts;
SELECT * FROM Employee;  -- shows 0 rows

-- Add NOT NULL and PK constraints
ALTER TABLE Employee MODIFY (EMPNO NOT NULL);
ALTER TABLE Employee ADD CONSTRAINT pk_emp PRIMARY KEY (EMPNO);
ALTER TABLE Employee MODIFY (ENAME NOT NULL, JOB NOT NULL, SAL NOT NULL);

-- Attempt to insert NULLs (will fail)
BEGIN
  INSERT INTO Employee VALUES (NULL,NULL,NULL,NULL,NULL,NULL);
  COMMIT;
EXCEPTION
  WHEN OTHERS THEN
    DBMS_OUTPUT.PUT_LINE('Error: '||SQLERRM);
    ROLLBACK;
END;
/


### Experiment 2

Create a table called Employee that contains attributes EMPNO,ENAME,JOB, 
MGR,SAL and execute the following. 
1. Add a column commission with domain to the Employee table. 
2. Insert any five records into the table. 
3. Update the column details of job. 
4. Rename the column of Employ table using alter command. 
5. Delete the employee whose EMPNO is 105.

In [ ]:
-- Create Employee table
CREATE TABLE Employee (
  EMPNO NUMBER,       -- employee number
  ENAME VARCHAR2(50), -- employee name
  JOB   VARCHAR2(30), -- job title
  MGR   NUMBER,       -- manager’s EMPNO
  SAL   NUMBER(10,2)  -- salary
);

-- 1. Add commission column
ALTER TABLE Employee
  ADD commission NUMBER(10,2);  -- commission amount

-- 2. Insert five records
INSERT INTO Employee VALUES (101, 'Alice',   'Developer', 100, 60000, 1000);
INSERT INTO Employee VALUES (102, 'Bob',     'Analyst',   100, 55000,  500);
INSERT INTO Employee VALUES (103, 'Carol',   'Developer', 101, 62000, 1200);
INSERT INTO Employee VALUES (104, 'Dave',    'Clerk',     102, 40000,   200);
INSERT INTO Employee VALUES (105, 'Eve',     'Analyst',   102, 58000,  750);

-- 3. Update job details (promote developers)
UPDATE Employee
  SET JOB = 'Senior Developer'
  WHERE JOB = 'Developer';

-- 4. Rename MGR column to MANAGER_ID
ALTER TABLE Employee
  RENAME COLUMN MGR TO MANAGER_ID;

-- 5. Delete employee with EMPNO = 105
DELETE FROM Employee
  WHERE EMPNO = 105;


### Experiment 3

Queries using aggregate functions ( COUNT, AVG, MIN, MAX, SUM), Group by, 
Order by. 
Employee(E_id, E_name, Age, Salary) 
1. Create Employee table containing all Records E_id, E_name, Age, Salary. 
2. Count number of employee names from employee table. 
3. Find the Maximum age from the employee table. 
4. Find the Minimum age from the employee table. 
5. Find salaries of employees in Ascending Order. 
6. Find grouped salaries of employees.

In [ ]:
-- 1. Create Employee table
CREATE TABLE Employee (
  E_id    NUMBER,        -- employee ID
  E_name  VARCHAR2(50),  -- employee name
  Age     NUMBER,        -- employee age
  Salary  NUMBER(10,2)   -- employee salary
);

-- 2. Count number of employee names
SELECT COUNT(E_name) AS name_count
  FROM Employee;

-- 3. Find the maximum age
SELECT MAX(Age) AS max_age
  FROM Employee;

-- 4. Find the minimum age
SELECT MIN(Age) AS min_age
  FROM Employee;

-- 5. List salaries in ascending order
SELECT Salary
  FROM Employee
  ORDER BY Salary ASC;

-- 6. Group employees by salary and count per salary
SELECT Salary,
       COUNT(*) AS employee_count
  FROM Employee
 GROUP BY Salary;


### Experiment 4

Create a row level trigger for the customers table that would fire for INSERT or UPDATE or DELETE operations performed on the CUSTOMERS table. This trigger will display the salary difference between the old & new Salary. CUSTOMERS (ID, NAME, AGE, ADDRESS, SALARY)

### Experiment 5

Create cursor for Employee table and extract the values from the table. Declare the variables, Open the cursor, and extract the values from the cursor. Close the cursor. Employee (E_id, E_name, Age, Salary)

In [ ]:
CREATE TABLE IF NOT EXISTS Employee (
  E_id    INT PRIMARY KEY,
  E_name  VARCHAR(50),
  Age     INT,
  Salary  DECIMAL(10,2)
);

INSERT INTO Employee VALUES
  (1, 'Alice',   30, 60000.00),
  (2, 'Bob',     25, 50000.00),
  (3, 'Charlie', 28, 55000.00);


DELIMITER $$

CREATE PROCEDURE print_employee_cursor()
BEGIN
  DECLARE v_id INT;
  DECLARE v_name VARCHAR(50);
  DECLARE v_age INT;
  DECLARE v_sal DECIMAL(10,2);
  DECLARE done INT DEFAULT 0;
  
  DECLARE emp_cur CURSOR FOR
    SELECT E_id, E_name, Age, Salary FROM Employee;
  DECLARE CONTINUE HANDLER FOR NOT FOUND SET done = 1;
  
  OPEN emp_cur;
  read_loop: LOOP
    FETCH emp_cur INTO v_id, v_name, v_age, v_sal;
    IF done THEN
      LEAVE read_loop;
    END IF;
    SELECT CONCAT(
      'ID=', v_id,
      ' Name=', v_name,
      ' Age=', v_age,
      ' Salary=', v_sal
    ) AS row_data;
  END LOOP;
  CLOSE emp_cur;
END$$

DELIMITER ;

-- To run:
CALL print_employee_cursor();


### Experiment 6

Install an Open-Source NoSQL Data base MongoDB & perform basic CRUD (Create, Read, Update & Delete) operations. Execute MongoDB basic Queries using CRUD operations.

In [ ]:
// Switch to (or create) the database
use companyDB

// ==== CREATE ====
// Insert a single document
db.employees.insertOne({
  empno: 101,
  name: "Alice",
  job: "Developer",
  salary: 60000
})

// Insert multiple documents
db.employees.insertMany([
  { empno: 102, name: "Bob",   job: "Analyst",   salary: 55000 },
  { empno: 103, name: "Carol", job: "Manager",   salary: 75000 },
  { empno: 104, name: "Dave",  job: "Clerk",     salary: 40000 }
])

// ==== READ ====
// Find all documents
print("All employees:")
db.employees.find().pretty()

// Find one by empno
print("Employee 103:")
db.employees.findOne({ empno: 103 })

// ==== UPDATE ====
// Give Alice a raise and title change
db.employees.updateOne(
  { empno: 101 },
  { $set: { job: "Senior Developer", salary: 65000 } }
)

// Increase salary by 10% for all Analysts
db.employees.updateMany(
  { job: "Analyst" },
  { $mul: { salary: 1.10 } }
)

// Show updates
print("After updates:")
db.employees.find().pretty()

// ==== DELETE ====
// Remove Dave
db.employees.deleteOne({ empno: 104 })

// Remove anyone earning below 50000
db.employees.deleteMany({ salary: { $lt: 50000 } })

// Final state
print("After deletes:")
db.employees.find().pretty()
